In [ ]:
#Mount Drive

from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import cv2 as cv
import numpy as np
import glob
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions
from numpy import savetxt


In [ ]:

videoid2 = 1
videoid = 0
sample = 0
sample2 = 0
magnitudeList = list()
orientationList = list()
magorientList = []
magorientVector = []
magorient3DVector = []
labelArray = []
path = ('/path/*.*')  #Give a path for specific folder to read videos
video_list = []
VideoCounter = 0   #Video counter value holds number of videos


def videopath():  #In this  code segment videos is sending one by one to opticalflow function
    global VideoCounter
    i = 0
    for video in glob.glob(path):
        a = cv.imread(video)
        video_list.insert(VideoCounter, video)
        VideoCounter = VideoCounter + 1
    while i != VideoCounter:
        opticalflow(video_list[i])
        i = i + 1

textfile = open("Magnitude.txt", "w")
textfile1 = open("Orientation.txt", "w")
textfile2 = open("Megorientation.txt", "w")


def opticalflow(video):
    global videoid
    print(video)
    capture = cv.VideoCapture(video)  #Capture the video
    ret, frame = capture.read()       #Getting frames from the video
    prvs = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)    
    hsv = np.zeros_like(frame)
    hsv[..., 1] = 244
    frameCounter = 0   #frame counter holds the value that frame we have
    counter = 0
    videoid = videoid + 1
    try:  
        while(1):
            ret = capture.grab()  # grabing the frame
            frameCounter = frameCounter + 1  # increment counter
            if frameCounter % 5 == 2:  # # display and detect only fifth of the frames,
                counter = counter + 1
                ret, frame = capture.retrieve()  # decoding the frame
                if ret:
                    ret2, frame2 = capture.read()
                    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
                    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])  # Extracting the attributes magnitude and angle (orientation)
                    hsv[..., 0] = ang * 180 / np.pi / 2
                    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
                    rgb = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
                    systemVertical = np.concatenate((frame2, rgb), axis=1)
                    #cv.imshow('opticalflow', systemVertical)                                 
                    magnitudeList = np.asarray(mag)
                    orientationList = np.asarray(ang)
                    magorientList = mag *ang   # magOrientedList holds the value that multiplication of attributes(mag and orientation)
                    magorientList = np.resize(magorientList, (224, 224, 3) )  #Resize the magnitude and angle to function vGG16
                    print('MAGORIENTED: ', magorientList)
                    transformedImage = np.expand_dims(magorientList, axis = 0)               
                    textfile.write(str(mag))                   
                    textfile1.write(str(ang))                    #Creating the text file for each attribute and their multiplication
                    textfile2.write(str(mag*ang))                  
                    print(counter)
                    vGG16(magorientList, videoid)    #Function call for vGG16
                    prvs = next
                    if cv.waitKey(1) & 0xFF == ord('q'):
                        break
                else:
                    return

    except:
        capture.release()
        cv.destroyAllWindows()

def vGG16(magorientList, videoid):
    global magorientVector
    global videoid2
    global sample
    global sample2
    sample = sample + 1
    if(videoid == videoid2):     
        model = VGG16(weights='imagenet', include_top=True)                  #model call vGG16 for transferLearning
        transformedImage = np.expand_dims(magorientList, axis = 0)      #To add additional dimension for keras application
        print(transformedImage.shape)
        transformedImage = preprocess_input(transformedImage)    #preprocess_input function properly transforms a standard matrice into the format which model requires.
        print(transformedImage)       #print matrices
        prediction = model.predict(transformedImage)      #predict() function classify input matrices in 1000 possible classes.
        print('prediction shape', prediction.shape)
        print('prediction', prediction)
        magorientVector.append(prediction)       #Append the vector list to magorientVector
        if(sample % 30 == 0):                  
            sample2 = sample2 + 1
            np_magorientVector = np.array(magorientVector)   #magorientVector is a 3dvector holds that every 30 samples of the fifth frames
            magorient3DVector.append(np_magorientVector)
            magorientVector.clear()
            transformedImage2 = np.expand_dims(magorient3DVector, axis = 0)
            print(transformedImage2.shape)
    else:
      sample = 0
      videoid2 = videoid
      magorientVector.clear()

videopath()

In [ ]:
np_magorient3DVector = np.asarray(magorient3DVector)
np_magorient3DVector.shape
np_magorient3DVector = np.squeeze(np_magorient3DVector)

In [ ]:
import numpy
import numpy as gfg

np_magorient3DVector_reshaped = np_magorient3DVector.reshape(np_magorient3DVector.shape[0], -1)
gfg.savetxt("Dataset.txt", np_magorient3DVector_reshaped)
loaded_newArray = gfg.loadtxt("Dataset.txt")

load_original_newArray = loaded_newArray.reshape(loaded_newArray.shape[0], loaded_newArray.shape[1] // np_magorient3DVector.shape[2], np_magorient3DVector.shape[2])

print("shape of arr: ", np_magorient3DVector.shape)
print("shape of load_original_arr: ", load_original_newArray.shape)

if (load_original_newArray == np_newArray).all():
    print("Yes, both the arrays are same")
else:
    print("No, both the arrays are not same")

In [ ]:
import numpy as np
 
#fight1= open(, "r")
# Read the array from disk
text_fight_data = np.loadtxt("/path/")

# Note that this returned a 2D array!
print (text_fight_data.shape)

# However, going back to 3D is easy if we know the 
# original shape of the array
fight_3D_data = text_fight_data.reshape((1548,30,1000))
    
# Just to check that they're the same...
print(fight_3D_data)

In [ ]:
import numpy as np
 
#fight1= open(, "r")
# Read the array from disk
text_normal_data = np.loadtxt("/path/")

# Note that this returned a 2D array!
print (text_normal_data.shape)

# However, going back to 3D is easy if we know the 
# original shape of the array
normal6_3D_data = text_normal_data.reshape((784,30,1000))
print (normal_3D_data.shape)
    
# Just to check that they're the same...
print(normal_3D_data)

In [ ]:
data = np.vstack((fight_3D_data, normal_3D_data))

print(data.shape)

In [ ]:
import numpy
import numpy as gfg

data_reshaped = data.reshape(data.shape[0], -1)
gfg.savetxt("Dataset.txt", data_reshaped)

